In [2]:
pip install matplotlib seaborn

   ---------------------------------------- 0.0/8.0 MB ? eta -:--:--
   ----------- ---------------------------- 2.4/8.0 MB 11.2 MB/s eta 0:00:01
   ---------------------- ----------------- 4.5/8.0 MB 10.7 MB/s eta 0:00:01
   -------------------------------- ------- 6.6/8.0 MB 10.3 MB/s eta 0:00:01
   ---------------------------------------  7.9/8.0 MB 9.2 MB/s eta 0:00:01
   ---------------------------------------- 8.0/8.0 MB 8.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   -------------- ------------------------- 0.8/2.2 MB 4.2 MB/s eta 0:00:01
   --------------------------------- ------ 1.8/2.2 MB 4.6 MB/s eta 0:00:01
   ---------------------------------------- 2.2/2.2 MB 4.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc, precision_recall_fscore_support
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Load dataset
df = pd.read_csv("Binary.csv")

In [3]:
# Split into features (X) and target (y)
X = df.drop(columns=['target'])  # Drop target column
y = df['target'].values  # Extract target column

In [4]:
# Convert target into tensor
y = torch.tensor(y, dtype=torch.float32).view(-1, 1)  # Ensure y is 2D for BCE loss

In [5]:
# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [6]:
# Convert to tensors
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)

In [7]:
# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y, test_size=0.2, random_state=42)

In [8]:
# Define the Neural Network Model
class MLP(nn.Module):
    def __init__(self, input_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)  # First hidden layer
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(64, 32)  # Second hidden layer
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(32, 1)  # Output layer
        self.sigmoid = nn.Sigmoid()  # Binary classification activation

    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

In [9]:
# Initialize model
input_dim = X_train.shape[1]
model = MLP(input_size=input_dim)

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
X_train, X_test, y_train, y_test = X_train.to(device), X_test.to(device), y_train.to(device), y_test.to(device)

In [12]:
# Define loss and optimizer
criterion = nn.BCELoss()  # Binary Cross Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam Optimizer

In [ ]:
# Training the Neural Network
epochs = 100
batch_size = 32
losses = []

for epoch in range(epochs):
    model.train()  # Set model to training mode
    optimizer.zero_grad()
    
    # Forward pass
    y_pred = model(X_train)
    
    # Compute loss
    loss = criterion(y_pred, y_train)
    
    # Backward pass
    loss.backward()
    optimizer.step()
    
    # Store loss
    losses.append(loss.item())

    if epoch % 10 == 0:
        print(f"Epoch {epoch}/{epochs}, Loss: {loss.item()}")

In [ ]:
# Plot training loss
plt.plot(losses)
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Training Loss Curve")
plt.show()

In [ ]:
# Model Evaluation
model.eval()  # Set model to evaluation mode
with torch.no_grad():
    y_pred_test = model(X_test)

In [ ]:
# Convert probabilities to binary output (Threshold = 0.5)
y_pred_binary = (y_pred_test >= 0.5).float()

In [ ]:
# Compute metrics
accuracy = accuracy_score(y_test.numpy(), y_pred_binary.numpy())
precision, recall, f1, _ = precision_recall_fscore_support(y_test.numpy(), y_pred_binary.numpy(), average="binary")
cm = confusion_matrix(y_test.numpy(), y_pred_binary.numpy())


In [ ]:
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

In [ ]:
# Plot Confusion Matrix
plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=["Class 0", "Class 1"], yticklabels=["Class 0", "Class 1"])
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# ROC Curve
fpr, tpr, _ = roc_curve(y_test.numpy(), y_pred_test.numpy())
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(6,4))
plt.plot(fpr, tpr, color="blue", label=f"ROC Curve (AUC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], color="gray", linestyle="--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend()
plt.show()